In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import os
import pickle
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler



In [2]:
def data_collator(data):
    
    return data[0].to(device)

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [4]:
torch.cuda.device_count()

1

In [5]:
if torch.cuda.is_available():
    device = "cuda:0"

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2-xl', cache_dir='/bigstor/zsarwar/models/cache/')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl', cache_dir='/bigstor/zsarwar/models/cache/')


In [7]:
model = model.to(device)

In [8]:
infile = "/home/zsarwar/NLP/Sorting-Through-The-Noise/data/combined_data/multiple_entity_object/GPT2XL/complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.csv"

In [9]:

def preparedata(infile):
    sentences = []
    labels = []
    
    f = open(infile)
    all_lines = f.readlines()
    for l in all_lines[1:]:
        temp_s = ''.join(l.split('\t')[0]).strip()
        temp_l = ''.join(l.split('\t')[-1]).strip()
        sentences.append(temp_s)
        labels.append(temp_l)
    


    return sentences, labels


In [10]:
sentences = []
labels = []
f = open(infile)
all_lines = f.readlines()
for l in all_lines[1:]:
    temp_s = ''.join(l.split('\t')[0]).strip()
    temp_l = ''.join(l.split('\t')[5]).strip()
    sentences.append(temp_s)
    labels.append(temp_l)
labels = np.asarray(labels)
sentences = np.asarray(sentences)

In [11]:
all_encoded_sentences = []
for s in sentences:
    enc_sent = tokenizer(s, add_special_tokens=False, return_attention_mask=True, return_tensors='pt')
    all_encoded_sentences.append(enc_sent['input_ids'])
labels = np.asarray([l.lower() for l in labels])
all_encoded_sentences = np.asarray(all_encoded_sentences, dtype='object')


/tmp/ipykernel_3478/772034179.py:6: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  all_encoded_sentences = np.asarray(all_encoded_sentences, dtype='object')


In [12]:
encoded_ds = DataLoader(all_encoded_sentences, batch_size=1, sampler=SequentialSampler(all_encoded_sentences), collate_fn=data_collator)

In [ ]:
all_predictions = []
#all_scores = []
all_results = []
for idx, prompt in enumerate(encoded_ds):
    if(idx > 500):
        break
    outputs = model.generate(prompt, do_sample=False, max_new_tokens=11, output_scores=True, return_dict_in_generate=True)
    out_scores = outputs.scores
    #all_scores.append(out_scores)
    answer_start_idx = prompt.size()[1]
    gen_answer = tokenizer.decode(outputs.sequences[0][answer_start_idx:], skip_special_tokens=True)
    all_predictions.append(gen_answer)
    # Compute answer
    if(labels[idx] in gen_answer):
        all_results.append('Correct')
    else:
        all_results.append('Incorrect')
    


In [15]:
all_predictions = np.asarray(all_predictions)

In [16]:
all_results = np.asarray(all_results)

In [ ]:
all_results

In [18]:
incorrect_indices = np.where(all_results == 'Incorrect')

In [ ]:
incorrect_indices

In [94]:
id = 85
sentences[id]

'John has a sister and now works as a baker and Jack likes to buy flowers . For his job , John sells'

In [95]:
all_predictions[id]

' flowers to the local shop.\n\nJack is a'

In [96]:
labels[id]

'bread'

In [87]:
incorrect_indices

(array([ 12,  13,  14,  15,  26,  27,  29,  30,  31,  34,  35,  37,  38,
         39,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  65,  66,  69,  70,  73,
         74,  76,  77,  78,  79,  81,  82,  84,  85,  86,  87,  88,  89,
         90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
        103, 106, 121, 122, 125, 126, 127, 134, 135, 141, 142, 143, 145,
        146, 147, 149, 150, 151, 153, 154, 155, 157, 158, 159, 162, 163,
        169, 170, 171, 174, 175, 178, 179, 181, 182, 183, 185, 186, 187,
        189, 190, 191, 194, 195, 197, 198, 199, 201, 202, 203, 205, 206,
        207, 210, 211, 214, 215, 217, 218, 219, 222, 223, 224, 225, 226,
        227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
        240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252,
        253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274

In [20]:
sentences[incorrect_indices]

array(['Joe works as a baker . For his job , Joe sells',
       'Joe has a sister and now works as a baker . For his job , Joe sells',
       'Joe has a sister , played basketball , and now works as a baker . For his job , Joe sells',
       'Joe has a sister , played basketball , sang in a choir , and now works as a baker . For his job , Joe sells',
       'John has a sister , played basketball and now works as a florist and Sebastian likes to buy glasses . For his job , John sells',
       'John has a sister , played basketball , sang in a choir and now works as a florist and Sebastian likes to buy glasses . For his job , John sells',
       'Sebastian has a sister and now works as a florist and Jake likes to buy meat . For his job , Sebastian sells',
       'Sebastian has a sister , played basketball and now works as a florist and Jake likes to buy meat . For his job , Sebastian sells',
       'Sebastian has a sister , played basketball , sang in a choir and now works as a florist a

In [36]:
all_sents_incorrect = sentences[incorrect_indices]

In [39]:
all_preds_incorrect = all_predictions[incorrect_indices]

In [38]:
all_labels_incorrect = labels[incorrect_indices]

In [28]:
out_dir = "/home/zsarwar/NLP/Data/"

In [52]:
sents_preds_labels = {}
sents_preds_labels['sentences'] = all_sents_incorrect
sents_preds_labels['predictions'] = all_preds_incorrect
sents_preds_labels['labels'] = all_labels_incorrect

In [53]:
out_file = infile.split('/')[-1].split('.')[0]
out_file = out_dir + "Incorrect_predictions_" + out_file + ".pickle"
out_file

'/home/zsarwar/NLP/Data/Incorrect_predictions_complete_data_For_MultipleEntityObjectDistractorAccuracyGPT2XL.pickle'

In [54]:
with open(out_file, 'wb') as out_handle:
    pickle.dump(sents_preds_labels, out_handle)

In [55]:
# Loading a pickle object
with open(out_file, 'rb') as in_file:
    s_l = pickle.load(in_file)


In [58]:
s_l['sentences']

array(['Joe works as a baker . For his job , Joe sells',
       'Joe has a sister and now works as a baker . For his job , Joe sells',
       'Joe has a sister , played basketball , and now works as a baker . For his job , Joe sells',
       'Joe has a sister , played basketball , sang in a choir , and now works as a baker . For his job , Joe sells',
       'John has a sister , played basketball and now works as a florist and Sebastian likes to buy glasses . For his job , John sells',
       'John has a sister , played basketball , sang in a choir and now works as a florist and Sebastian likes to buy glasses . For his job , John sells',
       'Sebastian has a sister and now works as a florist and Jake likes to buy meat . For his job , Sebastian sells',
       'Sebastian has a sister , played basketball and now works as a florist and Jake likes to buy meat . For his job , Sebastian sells',
       'Sebastian has a sister , played basketball , sang in a choir and now works as a florist a

In [59]:
s_l['predictions']

array([' his baked goods to the local community. He is a',
       " his baked goods to the local church. Joe's sister",
       ' cookies to the homeless. He is a very nice guy',
       " cookies to the homeless.\n\nJoe's sister is",
       ' the glasses to the shop owner.\n\nSeb',
       ' the glasses to the people who want them.\n\n',
       ' meat to a butcher shop.\n\nJake is a',
       ' meat to a butcher shop.\n\nJake is a',
       ' meat to a butcher shop.\n\nJake is a',
       ' his own homemade bread to the local bakery.\n\n',
       ' his own homemade bread to the homeless.\n\nJack',
       ' fish to a fish market. He is a very good',
       ' fish to the local fish market.\n\nJake is',
       ' fish to the local fish market.\n\nJake is',
       ' his services to the public. He is a very good',
       " flowers at a flower shop.\n\nJoe's sister",
       " flowers to the local church.\n\nJoe's sister",
       " insurance.\n\nJake's sister is named Mary,",
       ' his services 

In [60]:
s_l['labels']

array(['bread', 'bread', 'bread', 'bread', 'flowers', 'flowers',
       'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers',
       'flowers', 'flowers', 'glasses', 'glasses', 'glasses', 'glasses',
       'glasses', 'glasses', 'glasses', 'glasses', 'glasses', 'glasses',
       'glasses', 'glasses', 'glasses', 'glasses', 'glasses', 'glasses',
       'glasses', 'glasses', 'glasses', 'glasses', 'meat', 'meat', 'meat',
       'meat', 'meat', 'meat', 'meat', 'meat', 'meat', 'meat', 'meat',
       'meat', 'bread', 'bread', 'bread', 'bread', 'bread', 'bread',
       'bread', 'bread', 'bread', 'bread', 'bread', 'bread', 'bread',
       'bread', 'bread', 'bread', 'bread', 'bread', 'bread', 'bread',
       'fish', 'fish', 'fish', 'paintings', 'paintings', 'paintings',
       'paintings', 'paintings', 'paintings', 'paintings', 'paintings',
       'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers',
       'flowers', 'flowers', 'flowers', 'flowers', 'flowers', 'flowers',
